In [12]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = []
    for i = 1:size(X)[1]
        push!(L, intersect(X[i],Y[i]))
        # typeof(intersect(X[i],Y[i])) => Interval{Float64}
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=(0..0)
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end


function allsol(F,X)
    cnt=0
    L = [X] #調査する区間のリスト
    S = [] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        cnt+=1
        if cnt>10000
            break
        end
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(c)+j*(X-c)) )
            continue
        end
    
        tmp = ForwardDiff.jacobian(F,c)
        
        R = tmp
        try
            R = inv(tmp)
        catch e
            println(e)
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch e
                println("区間の分割に失敗")
                println(e)
                return false
            end
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        if (orig_is_empty(K,X))
            continue #解なし
        end
        
        if (orig_is_subset(K,X))　#解あり
            push!(S, K) #解のリストに追加
            continue
        end
        
        try
            X1,X2 = orig_devide(orig_intersect(K,X))　#K and X を2分割
            push!(L, X1)
            push!(L, X2)
        catch e
            println("区間の分割に失敗")
            println(e)
            return false
        end
        
    end
    
    return S
end

X = [(-10.. 10),(-10 .. 10)]

#f((x1,x2))= x1^2 + x2^2 -1
#g((x1,x2)) = x1-x2

f((x1,x2)) = x1 + 2*x2 +1
g((x1,x2)) = x1^2 + 2*(x2^2) -1

F((x,y))=[f((x,y));g((x,y))]

li=allsol(F,X)

for i in li
    @show i
end

# 境界に解がのっているとき、区間拡張？
# 重解　＝＞区間幅に制限をつける

LAPACKException(2)
SingularException(2)
SingularException(2)
SingularException(2)
SingularException(2)
SingularException(2)
i = Interval{Float64}[[0.158068, 0.527998], [-0.763999, -0.579034]]
